# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

Hi, Run the cell below and it will automatic install all the dependecies for PySyft. Use this Google Colab Workspace for your use 

Thank You

If you come across any issues, then feel free to contact me on Slack. My username is archit

In [0]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.1MB 2.8MB/s 
     |████████████████████████████████| 276kB 43.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 29220, done.
remote: Total 29220 (delta 0), reused 0 (delta 0), pack-reused 29220
Receiving objects: 100% (29220/29220), 32.18 MiB | 20.40 MiB/s, done.
Resolving deltas: 100% (19444/19444), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 2.9MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.1.21a1 has requirement tf_encrypted!=0.5.7,>=0.5.4, b

In [0]:
import torch as th

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [0]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
import syft as sy

W0730 01:53:24.375332 140102891689856 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0730 01:53:24.396337 140102891689856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [0]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [0]:
bob._objects

{45846175536: tensor([1, 2, 3, 4, 5])}

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.id_at_location

45846175536

In [0]:
x.id

52464434939

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:52464434939 -> bob:45846175536]

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [0]:
x_ptr.child.child

{'alice': [PointerTensor | me:85755807939 -> alice:84972296870],
 'bob': [PointerTensor | me:22206451469 -> bob:82539785738]}

In [0]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:22206451469 -> bob:82539785738]
	-> [PointerTensor | me:85755807939 -> alice:84972296870]

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:3602690192 -> bob:30680110637]

In [0]:
y

(Wrapper)>[PointerTensor | me:22314341344 -> bob:3905904494]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:66407641687 -> bob:68798766654]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:46665733596 -> bob:34694139169]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:88629103349 -> bob:36744000871]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [0]:
for i in range(10):
    
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
